In [1]:
import numpy
import matplotlib.pyplot as plt
from PIL import Image
import sklearn

In [2]:
def _get_image_as_array(path: str):
    im = Image.open(path)
    im = im.convert(mode="L")
    
    arr = numpy.asarray(im)

    return arr


In [3]:
def _transform_complexArray_to_real(i_arr: numpy.ndarray):
    ret = numpy.absolute(i_arr)
    return ret

In [4]:
def _high_pass_filter(arr: numpy.ndarray, radius: float):
    ret = numpy.zeros((arr.shape[0],arr.shape[1]),dtype=complex)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            if (i - arr.shape[0]/2)**2 + (j - arr.shape[1]/2)**2  - radius**2 < 0:
            #if i > arr.shape[0]*radius and j > arr.shape[1]*radius:
            #if (i - arr.shape[0]/2)**2 + (j - arr.shape[1]/2)**2 - radius**2 > 0:
                ret[i][j] = arr[i][j]
            else:
                ret[i][j] = 0
    return ret


In [5]:
def _fft_edge_detector(path: str, radius: float, show: bool):
    image_arr = _get_image_as_array(path)
    #plt.figure("original")
    #plt.imshow(image_arr, interpolation='none')
    #plt.title("Original image")

    image_fft = numpy.fft.fft2(image_arr)
    #plt.figure("dft of image")
    #plt.imshow(_transform_complexArray_to_real(image_fft), interpolation='none')
    #plt.title("dft of image")
    if show:
        im = Image.fromarray(_transform_complexArray_to_real(image_fft))
        im.show()

    fft_filtered = _high_pass_filter(image_fft, radius)
    #plt.figure("filtered dft")
    #plt.imshow(_transform_complexArray_to_real(fft_filtered), interpolation='none')
    #plt.title("filtered dft")
    if show:
        im = Image.fromarray(_transform_complexArray_to_real(fft_filtered))
        im.show()

    filtered_Image = numpy.fft.ifft2(fft_filtered)
    #plt.figure(radius)
    #plt.imshow(_transform_complexArray_to_real(filtered_Image), interpolation='none')
    #plt.title(f"Edge detected {radius}")
    if show:
        im = Image.fromarray(_transform_complexArray_to_real(filtered_Image))
        im.show()
    return _transform_complexArray_to_real(filtered_Image)



In [6]:
# r = 30 works good
_fft_edge_detector("sample/0.jpg",30, False)

array([[17.77699843, 24.75465755, 29.26102478, ..., 15.03630132,
        17.51502582, 16.23425728],
       [ 9.96779005,  7.2033433 ,  5.03003993, ...,  9.70780434,
         7.37657884, 11.15663327],
       [ 9.10997712, 15.1356791 ,  2.40186936, ..., 14.89723661,
         3.86507596,  6.95489447],
       ...,
       [33.80703327,  8.06005753, 10.77794106, ...,  0.36463699,
        11.28561539, 16.50635342],
       [20.14260652, 25.05587529, 22.13826323, ..., 35.63724605,
         5.04147975, 12.41605301],
       [22.44343853, 20.63012736, 29.19977843, ..., 35.35803542,
        45.9099621 , 24.96363455]])

In [7]:
import sklearn
import os

In [8]:
X = []
Y = []

cls  = os.listdir("data/train/")

for i in range(len(cls)):
    imgs = os.listdir("data/train/"+cls[i])
    for img in imgs:
        path = "data/train/"+ "/" + cls[i] + "/" + img
        im = _fft_edge_detector(path, 30, False)
        X.append(im.flatten())
        Y.append(i)
    

In [ ]:
X = numpy.concatenate(X)
Y = numpy.asarray(Y)

In [24]:
from sklearn.decomposition import PCA

pca = PCA(n_components='mle')
pca.fit(X,Y)
X_new = pca.transform(X)

In [25]:
numpy.save('X_pca_result', X_new)

In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC())
clf.fit(X_new, Y)

In [21]:



x_pred = _fft_edge_detector("data/validation/angry/5010.jpg", 30, False).flatten()
x_pred = pca.transform([x_pred])
p = clf.predict([x_pred])

clss = os.listdir("data/train/")

print(p)

[5]


In [22]:
import pickle
sav_non_pca = 'svm_classifier_with_pca'
pickle.dump(clf,open(sav_non_pca,'wb'))